In [1]:
!pip install segmentation-models-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 23.5 MB/s eta 0:00:00


In [3]:
!pip install ttach

In [4]:
!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.5-py3-none-any.whl size=44283 sha256=79277a029fafc4b9e7f603b65a4f4d73cb7926ecbb64634c2e7cf2047aab85f1
  Stored in directory: /root/.cache/pip/wheels/bc/52/78/893c3b94279ef238f43a9e89608af648de401b96415bebbd1f
Successfully built grad-cam


In [5]:
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image,deprocess_image
from pytorch_grad_cam.utils.find_layers import find_layer_predicate_recursive

In [6]:
import pandas as pd
from tabulate import tabulate

In [7]:
# Import the required packages

import torch
import argparse
import os
import cv2

from segmentation_models_pytorch import Unet

from skimage.io import imread,imsave
import numpy as np
import rasterio as rio
import geopandas as gpd
import matplotlib.pyplot as plt
import imageio

from utils import *
from color_map import cm_data
from rasterio.features import shapes
from tqdm.notebook import tqdm

In [8]:
# load your model with pre-trained model weights

MODEL_PATH = './pretrained_model'
THRESH = 0.5
ALPHA = 0.85
SCALE = None


model = Unet(
        encoder_name = "tu-tf_efficientnet_b0",
        encoder_depth= 5,
        encoder_weights = None,
        decoder_use_batchnorm = True,
        decoder_channels = (256, 128, 64, 32, 16),
        decoder_attention_type = None,
        in_channels= 3,
        classes = 3,
        activation = 'sigmoid',
        aux_params = None,
    )

model = load_model(model,MODEL_PATH)
# model.cuda()

In [9]:
# Define the CAM-based Extensions classes
from PIL import Image
import numpy as np
import ttach as tta
import sys
import torch
import warnings
from skimage.io import imread
import matplotlib.pyplot as plt
from segmentation_models_pytorch import Unet
import argparse
import os
from typing import Callable, List
import cv2
import tqdm
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
from skimage.segmentation import watershed
from skimage.measure import label
from pytorch_grad_cam.utils.svd_on_activations import get_2d_projection
from pytorch_grad_cam.utils.image import scale_cam_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.find_layers import replace_layer_recursive

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


# Grad-Cam Classes
class ActivationsAndGradients:
    """ Class for extracting activations and
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers, reshape_transform):
        self.model = model
        self.gradients = []
        self.activations = []
        self.reshape_transform = reshape_transform
        self.handles = []
        for target_layer in target_layers:
            self.handles.append(
                target_layer.register_forward_hook(self.save_activation))
            # Because of https://github.com/pytorch/pytorch/issues/61519,
            # we don't use backward hook to record gradients.
            self.handles.append(
                target_layer.register_forward_hook(self.save_gradient))

    def save_activation(self, module, input, output):
        activation = output
        if self.reshape_transform is not None:
            activation = self.reshape_transform(activation)
        self.activations.append(activation.cpu().detach())

    def save_gradient(self, module, input, output):
        if not hasattr(output, "requires_grad") or not output.requires_grad:
            # You can only register hooks on tensor requires grad.
            return

        # Gradients are computed in reverse order
        def _store_grad(grad):
            if self.reshape_transform is not None:
                grad = self.reshape_transform(grad)
            self.gradients = [grad.cpu().detach()] + self.gradients
            # self.gradients = [torch.mul(t, -1) for t in self.gradients]

        output.register_hook(_store_grad)

    def __call__(self, x):
        self.gradients = []
        self.activations = []
        return self.model(x)

    def release(self):
        for handle in self.handles:
            handle.remove()




class AblationLayer(torch.nn.Module):
    def __init__(self):
        super(AblationLayer, self).__init__()

    def objectiveness_mask_from_svd(self, activations, threshold=0.01):
        """ Experimental method to get a binary mask to compare if the activation is worth ablating.
            The idea is to apply the EigenCAM method by doing PCA on the activations.
            Then we create a binary mask by comparing to a low threshold.
            Areas that are masked out, are probably not interesting anyway.
        """

        projection = get_2d_projection(activations[None, :])[0, :]
        projection = np.abs(projection)
        projection = projection - projection.min()
        projection = projection / projection.max()
        projection = projection > threshold
        return projection

    def activations_to_be_ablated(
            self,
            activations,
            ratio_channels_to_ablate=1.0):
        """ Experimental method to get a binary mask to compare if the activation is worth ablating.
            Create a binary CAM mask with objectiveness_mask_from_svd.
            Score each Activation channel, by seeing how much of its values are inside the mask.
            Then keep the top channels.

        """
        if ratio_channels_to_ablate == 1.0:
            self.indices = np.int32(range(activations.shape[0]))
            return self.indices

        projection = self.objectiveness_mask_from_svd(activations)

        scores = []
        for channel in activations:
            normalized = np.abs(channel)
            normalized = normalized - normalized.min()
            normalized = normalized / np.max(normalized)
            score = (projection * normalized).sum() / normalized.sum()
            scores.append(score)
        scores = np.float32(scores)

        indices = list(np.argsort(scores))
        high_score_indices = indices[::-
                                     1][: int(len(indices) *
                                              ratio_channels_to_ablate)]
        low_score_indices = indices[: int(
            len(indices) * ratio_channels_to_ablate)]
        self.indices = np.int32(high_score_indices + low_score_indices)
        return self.indices

    def set_next_batch(
            self,
            input_batch_index,
            activations,
            num_channels_to_ablate):
        """ This creates the next batch of activations from the layer.
            Just take corresponding batch member from activations, and repeat it num_channels_to_ablate times.
        """
        self.activations = activations[input_batch_index, :, :, :].clone(
        ).unsqueeze(0).repeat(num_channels_to_ablate, 1, 1, 1)

    def __call__(self, x, test=None):
        output = self.activations
        for i in range(output.size(0)):
            # Commonly the minimum activation will be 0,
            # And then it makes sense to zero it out.
            # However depending on the architecture,
            # If the values can be negative, we use very negative values
            # to perform the ablation, deviating from the paper.
            if torch.min(output) == 0:
                output[i, self.indices[i], :] = 0
            else:
                ABLATION_VALUE = 1e7
                output[i, self.indices[i], :] = torch.min(
                    output) - ABLATION_VALUE

        return output




class GRADCAM_Extensions:
    def __init__(self, extension, model: torch.nn.Module, target_layers: List[torch.nn.Module], use_cuda: bool = False,
                 reshape_transform: Callable = None, compute_input_gradient: bool = False,
                 uses_gradients: bool = True) -> None:
        self.model = model.eval()
        self.target_layers = target_layers
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()
        self.reshape_transform = reshape_transform
        self.compute_input_gradient = compute_input_gradient
        self.uses_gradients = uses_gradients
        self.activations_and_grads = ActivationsAndGradients(self.model, target_layers, reshape_transform)
        self.extension = extension

    def forward(self, input_tensor: torch.Tensor, targets: List[torch.nn.Module],
                eigen_smooth: bool = False) -> np.ndarray:

        # print('\n----------------------------------------Seg-Grad-Cam----------------------------------------------------\n')

        if self.cuda:
            input_tensor = input_tensor.cuda()
        if self.compute_input_gradient:
            input_tensor = torch.autograd.Variable(input_tensor, requires_grad=True)

        outputs = self.activations_and_grads(input_tensor)
        if targets is None:
            target_categories = np.argmax(outputs.cpu().data.numpy(), axis=-1)
            targets = [ClassifierOutputTarget(category) for category in target_categories]

        if self.uses_gradients:
            self.model.zero_grad()
            loss = sum([target(output) for target, output in zip(targets, outputs)])
            loss.backward(retain_graph=True)

        activations_list = [a.cpu().data.numpy() for a in self.activations_and_grads.activations]
        grads_list = [g.cpu().data.numpy() for g in self.activations_and_grads.gradients]
        target_size = input_tensor.size(-1), input_tensor.size(-2)
        # print('Activations list size: ', len(activations_list))
        # print('Gradients list size: ', len(grads_list))
        # print('Target layer size: ', len(self.target_layers))

        cam_per_target_layer = []
        # Loop over the saliency image from every layer
        for i in range(len(self.target_layers)):
            target_layer = self.target_layers[i]
            # print('\t\t\t-----------------------\n')
            # print('Target Layer ', i + 1, ': ', target_layer)
            layer_activations = None
            layer_grads = None
            if i < len(activations_list):
                layer_activations = activations_list[i]
            if i < len(grads_list):
                layer_grads = grads_list[i]


            if  self.extension == "grad_cam":
                weights = np.mean(layer_grads, axis=(2, 3))
                weighs_up = weights[:, :, None, None]
                weighted_activations = weights[:, :, None, None] * layer_activations
                if eigen_smooth:
                    cam = get_2d_projection(weighted_activations)
                    print("Cam image per layer size: ", cam.shape)
                else:
                    cam = weighted_activations.sum(axis=1)

            elif self.extension == "hires_cam":
                elementwise_activations = layer_grads * layer_activations
                if eigen_smooth:
                    cam = get_2d_projection(elementwise_activations)
                else:
                    cam = elementwise_activations.sum(axis=1)

            elif self.extension == "ew_cam":
                elementwise_activations = np.maximum(layer_grads * layer_activations, 0)
                if eigen_smooth:
                    cam = get_2d_projection(elementwise_activations)
                else:
                    cam = elementwise_activations.sum(axis=1)

            elif self.extension == "grad_cam_pp":
                grads_power_2 = layer_grads**2
                grads_power_3 = grads_power_2 * layer_grads
                # Equation 19 in https://arxiv.org/abs/1710.11063
                sum_activations = np.sum(layer_activations, axis=(2, 3))
                eps = 0.000001
                aij = grads_power_2 / (2 * grads_power_2 + sum_activations[:, :, None, None] * grads_power_3 + eps)
                # Now bring back the ReLU from eq.7 in the paper,
                # And zero out aijs where the activations are 0
                aij = np.where(layer_grads != 0, aij, 0)
                weights = np.maximum(layer_grads, 0) * aij
                weights = np.sum(weights, axis=(2, 3))
                weighted_activations = weights[:, :, None, None] * layer_activations
                if eigen_smooth:
                    cam = get_2d_projection(weighted_activations)
                    print("Cam image per layer size: ", cam.shape)
                else:
                    cam = weighted_activations.sum(axis=1)



            elif self.extension == "x_grad_cam":
                sum_activations = np.sum(layer_activations, axis=(2, 3))
                eps = 1e-7
                weights = layer_grads * layer_activations / \
                (sum_activations[:, :, None, None] + eps)
                weights = weights.sum(axis=(2, 3))
                weighted_activations = weights[:, :, None, None] * layer_activations
                if eigen_smooth:
                    cam = get_2d_projection(weighted_activations)
                    print("Cam image per layer size: ", cam.shape)
                else:
                    cam = weighted_activations.sum(axis=1)


            elif self.extension == "score_cam":
                with torch.no_grad():
                    upsample = torch.nn.UpsamplingBilinear2d(size=input_tensor.shape[-2:])
                    activation_tensor = torch.from_numpy(layer_activations)
                    if self.cuda:
                        activation_tensor = activation_tensor.cuda()
                    upsampled = upsample(activation_tensor)
                    maxs = upsampled.view(upsampled.size(0), upsampled.size(1), -1).max(dim=-1)[0]
                    mins = upsampled.view(upsampled.size(0), upsampled.size(1), -1).min(dim=-1)[0]
                    maxs, mins = maxs[:, :, None, None], mins[:, :, None, None]
                    upsampled = (upsampled - mins) / (maxs - mins)

                    input_tensors = input_tensor[:, None, :, :] * upsampled[:, :, None, :, :]
                    if hasattr(self, "batch_size"):
                        BATCH_SIZE = self.batch_size
                    else:
                        BATCH_SIZE = 8


                    scores = []
                    for target, tensor in zip(targets, input_tensors):
                        for i in tqdm.tqdm(range(0, tensor.size(0), BATCH_SIZE)):
                            batch = tensor[i: i + BATCH_SIZE, :]
                            outputs = [target(o).cpu().item() for o in self.model(batch)]
                            scores.extend(outputs)
                    scores = torch.Tensor(scores)
                    scores = scores.view(layer_activations.shape[0], layer_activations.shape[1])
                    weights = torch.nn.Softmax(dim=-1)(scores).numpy()

                weighted_activations = weights[:, :, None, None] * layer_activations
                if eigen_smooth:
                    cam = get_2d_projection(weighted_activations)
                    print("Cam image per layer size: ", cam.shape)
                else:
                    cam = weighted_activations.sum(axis=1)

            elif self.extension == "layer_cam":
                spatial_weighted_activations = np.maximum(layer_grads, 0) * layer_activations
                if eigen_smooth:
                    cam = get_2d_projection(spatial_weighted_activations)
                else:
                    cam = spatial_weighted_activations.sum(axis=1)


            elif self.extension == "eigen_cam":
                cam = get_2d_projection(layer_activations)

            elif self.extension == "eigen_grad_cam":
                cam = get_2d_projection(layer_grads * layer_activations)

            else:
                print("Unkown Extension. Please use one of the following: grad_cam - hires_cam - ew_cam -  grad_cam_pp - x_grad_cam - score_cam - layer_cam - eigen_cam -  eigen_grad_cam")



            cam = np.maximum(cam, 0)
            # print("Cam image  Max per layer size: ", cam.shape)
            scaled = scale_cam_image(cam, target_size)
            # print("Scaled Cam image per layer size: ", scaled.shape)
            cam_per_target_layer.append(scaled[:, None, :])

        # print("Cam image list size: ", len(cam_per_target_layer))
        cam_per_target_layer = np.concatenate(cam_per_target_layer, axis=1)
        # print("Cam image list Concat size: ", len(cam_per_target_layer))
        cam_per_target_layer = np.maximum(cam_per_target_layer, 0)
        # print("Cam image list (max) size: ", len(cam_per_target_layer))
        result = np.mean(cam_per_target_layer, axis=1) # old: mean
        # print("+++ Averaged CAM list size: ", result.shape)


        return scale_cam_image(result) # result

    def __call__(self, input_tensor: torch.Tensor, targets: List[torch.nn.Module] = None, aug_smooth: bool = False,
                 eigen_smooth: bool = False) -> np.ndarray:
        # Smooth the CAM result with test time augmentation
        if aug_smooth is True:
            transforms = tta.Compose(
                [
                    tta.HorizontalFlip(),
                    tta.Multiply(factors=[0.9, 1, 1.1]),
                ]
            )
            cams = []
            for transform in transforms:
                augmented_tensor = transform.augment_image(input_tensor)
                cam = self.forward(augmented_tensor, targets, eigen_smooth)
                # The ttach library expects a tensor of size BxCxHxW
                cam = cam[:, None, :, :]
                cam = torch.from_numpy(cam)
                cam = transform.deaugment_mask(cam)
                # Back to numpy float32, HxW
                cam = cam.numpy()
                cam = cam[:, 0, :, :]
                cams.append(cam)
            cam = np.mean(np.float32(cams), axis=0)
            return cam
        else:
            return self.forward(input_tensor, targets, eigen_smooth)

    def __del__(self):
        self.activations_and_grads.release()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.activations_and_grads.release()
        if isinstance(exc_value, IndexError):
            # Handle IndexError here...
            print(
                f"An exception occurred in CAM with block: {exc_type}. Message: {exc_value}")
            return True




class GRADCAMEXTENDED_AblationCAM:
    def __init__(self, model: torch.nn.Module, target_layers: List[torch.nn.Module], use_cuda: bool = False,
                 reshape_transform: Callable = None, compute_input_gradient: bool = False,
                 uses_gradients: bool = True,
                 ablation_layer: torch.nn.Module = AblationLayer(),
                 batch_size: int = 32,
                 ratio_channels_to_ablate: float = 1.0) -> None:
        self.model = model.eval()
        self.target_layers = target_layers
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()
        self.reshape_transform = reshape_transform
        self.compute_input_gradient = compute_input_gradient
        self.uses_gradients = uses_gradients
        self.activations_and_grads = ActivationsAndGradients(self.model, target_layers, reshape_transform)
        self.batch_size = batch_size
        self.ablation_layer = ablation_layer
        self.ratio_channels_to_ablate = ratio_channels_to_ablate


    def save_activation(self, module, input, output) -> None:
        """ Helper function to save the raw activations from the target layer """
        self.activations = output

    def assemble_ablation_scores(self,
                                 new_scores: list,
                                 original_score: float,
                                 ablated_channels: np.ndarray,
                                 number_of_channels: int) -> np.ndarray:
        """ Take the value from the channels that were ablated,
            and just set the original score for the channels that were skipped """

        index = 0
        result = []
        sorted_indices = np.argsort(ablated_channels)
        ablated_channels = ablated_channels[sorted_indices]
        new_scores = np.float32(new_scores)[sorted_indices]

        for i in range(number_of_channels):
            if index < len(ablated_channels) and ablated_channels[index] == i:
                weight = new_scores[index]
                index = index + 1
            else:
                weight = original_score
            result.append(weight)

        return result


    def forward(self, input_tensor: torch.Tensor, targets: List[torch.nn.Module],
                eigen_smooth: bool = False) -> np.ndarray:

        # print('\n----------------------------------------Seg-Grad-Cam----------------------------------------------------\n')

        if self.cuda:
            input_tensor = input_tensor.cuda()
        if self.compute_input_gradient:
            input_tensor = torch.autograd.Variable(input_tensor, requires_grad=True)

        outputs = self.activations_and_grads(input_tensor)
        if targets is None:
            target_categories = np.argmax(outputs.cpu().data.numpy(), axis=-1)
            targets = [ClassifierOutputTarget(category) for category in target_categories]

        if self.uses_gradients:
            self.model.zero_grad()
            loss = sum([target(output) for target, output in zip(targets, outputs)])
            loss.backward(retain_graph=True)

        activations_list = [a.cpu().data.numpy() for a in self.activations_and_grads.activations]
        grads_list = [g.cpu().data.numpy() for g in self.activations_and_grads.gradients]
        target_size = input_tensor.size(-1), input_tensor.size(-2)
        # print('Activations list size: ', len(activations_list))
        # print('Gradients list size: ', len(grads_list))
        # print('Target layer size: ', len(self.target_layers))

        cam_per_target_layer = []
        # Loop over the saliency image from every layer
        for i in range(len(self.target_layers)):
            target_layer = self.target_layers[i]
            # print('\t\t\t-----------------------\n')
            # print('Target Layer ', i + 1, ': ', target_layer)
            layer_activations = None
            layer_grads = None
            if i < len(activations_list):
                layer_activations = activations_list[i]
            if i < len(grads_list):
                layer_grads = grads_list[i]


            # get weights
            # Do a forward pass, compute the target scores, and cache the
            # activations
            handle = target_layer.register_forward_hook(self.save_activation)
            with torch.no_grad():
                outputs = self.model(input_tensor)
                handle.remove()
                original_scores = np.float32(
                    [target(output).cpu().item() for target, output in zip(targets, outputs)])

            # Replace the layer with the ablation layer.
            # When we finish, we will replace it back, so the original model is
            # unchanged.
            ablation_layer = self.ablation_layer
            replace_layer_recursive(self.model, target_layer, ablation_layer)

            number_of_channels = layer_activations.shape[1]
            weights = []
            # This is a "gradient free" method, so we don't need gradients here.
            with torch.no_grad():
                # Loop over each of the batch images and ablate activations for it.
                for batch_index, (target, tensor) in enumerate(
                        zip(targets, input_tensor)):
                    new_scores = []
                    batch_tensor = tensor.repeat(self.batch_size, 1, 1, 1)

                    # Check which channels should be ablated. Normally this will be all channels,
                    # But we can also try to speed this up by using a low
                    # ratio_channels_to_ablate.
                    channels_to_ablate = ablation_layer.activations_to_be_ablated(
                        layer_activations[batch_index, :], self.ratio_channels_to_ablate)
                    number_channels_to_ablate = len(channels_to_ablate)

                    for i in tqdm.tqdm(range(0, number_channels_to_ablate, self.batch_size)):
                        if i + self.batch_size > number_channels_to_ablate:
                            batch_tensor = batch_tensor[:(number_channels_to_ablate - i)]

                        # Change the state of the ablation layer so it ablates the next channels.
                        # TBD: Move this into the ablation layer forward pass.
                        ablation_layer.set_next_batch(input_batch_index=batch_index,activations=self.activations,
                            num_channels_to_ablate=batch_tensor.size(0))


                        score = [target(o).cpu().item() for o in self.model(batch_tensor)]
                        new_scores.extend(score)

                        ablation_layer.indices = ablation_layer.indices[batch_tensor.size(0):]


                    new_scores = self.assemble_ablation_scores(new_scores,original_scores[batch_index], channels_to_ablate,
                        number_of_channels)
                    weights.extend(new_scores)

            weights = np.float32(weights)
            weights = weights.reshape(layer_activations.shape[:2])
            original_scores = original_scores[:, None]
            weights = (original_scores - weights) / original_scores

            # Replace the model back to the original state
            #-----------------------------------
            replace_layer_recursive(self.model, ablation_layer, target_layer)


            # Equation 3.1
            weighted_activations = weights[:, :, None, None] * layer_activations

            # Equation 3.2
            if eigen_smooth:
                cam = get_2d_projection(weighted_activations)
            else:
                cam = weighted_activations.sum(axis=1)

            cam = np.maximum(cam, 0)
            # print("Cam image  Max per layer size: ", cam.shape)
            scaled = scale_cam_image(cam, target_size)
            # print("Scaled Cam image per layer size: ", scaled.shape)
            cam_per_target_layer.append(scaled[:, None, :])

        # print("Cam image list size: ", len(cam_per_target_layer))
        cam_per_target_layer = np.concatenate(cam_per_target_layer, axis=1)
        # print("Cam image list Concat size: ", len(cam_per_target_layer))
        cam_per_target_layer = np.maximum(cam_per_target_layer, 0)
        # print("Cam image list (max) size: ", len(cam_per_target_layer))
        result = np.mean(cam_per_target_layer, axis=1) # old: mean
        # print("+++ Averaged CAM list size: ", result.shape)


        return scale_cam_image(result) # result

    def __call__(self, input_tensor: torch.Tensor, targets: List[torch.nn.Module] = None, aug_smooth: bool = False,
                 eigen_smooth: bool = False) -> np.ndarray:
        # Smooth the CAM result with test time augmentation
        if aug_smooth is True:
            transforms = tta.Compose(
                [
                    tta.HorizontalFlip(),
                    tta.Multiply(factors=[0.9, 1, 1.1]),
                ]
            )
            cams = []
            for transform in transforms:
                augmented_tensor = transform.augment_image(input_tensor)
                cam = self.forward(augmented_tensor, targets, eigen_smooth)
                # The ttach library expects a tensor of size BxCxHxW
                cam = cam[:, None, :, :]
                cam = torch.from_numpy(cam)
                cam = transform.deaugment_mask(cam)
                # Back to numpy float32, HxW
                cam = cam.numpy()
                cam = cam[:, 0, :, :]
                cams.append(cam)
            cam = np.mean(np.float32(cams), axis=0)
            return cam
        else:
            return self.forward(input_tensor, targets, eigen_smooth)

    def __del__(self):
        self.activations_and_grads.release()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.activations_and_grads.release()
        if isinstance(exc_value, IndexError):
            # Handle IndexError here...
            print(
                f"An exception occurred in CAM with block: {exc_type}. Message: {exc_value}")
            return True





from pytorch_grad_cam.utils.find_layers import find_layer_predicate_recursive
from pytorch_grad_cam.utils.image import scale_accross_batch_and_channels


class GRADCAMEXTENDED_FullGrad:
    def __init__(self, model: torch.nn.Module, target_layers: List[torch.nn.Module], use_cuda: bool = False,
                 reshape_transform: Callable = None, compute_input_gradient: bool = True,
                 uses_gradients: bool = True) -> None:

        if len(target_layers) > 0:
            print(
                "Warning: target_layers is ignored in FullGrad. All bias layers will be used instead")

        def layer_with_2D_bias(layer):
            bias_target_layers = [torch.nn.Conv2d, torch.nn.BatchNorm2d]
            if type(layer) in bias_target_layers and layer.bias is not None:
                return True
            return False
        target_layers = find_layer_predicate_recursive(model, layer_with_2D_bias)
        self.bias_data = [self.get_bias_data(layer).cpu().numpy() for layer in target_layers]




        self.model = model.eval()
        self.target_layers = target_layers
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()
        self.reshape_transform = reshape_transform
        self.compute_input_gradient = compute_input_gradient
        self.uses_gradients = uses_gradients
        self.activations_and_grads = ActivationsAndGradients(self.model, target_layers, reshape_transform)


    def get_bias_data(self, layer):
        # Borrowed from official paper impl:
        # https://github.com/idiap/fullgrad-saliency/blob/master/saliency/tensor_extractor.py#L47
        if isinstance(layer, torch.nn.BatchNorm2d):
            bias = - (layer.running_mean * layer.weight
                      / torch.sqrt(layer.running_var + layer.eps)) + layer.bias
            return bias.data
        else:
            return layer.bias.data

    def forward(self, input_tensor: torch.Tensor, targets: List[torch.nn.Module],
                eigen_smooth: bool = False) -> np.ndarray:

        # print('\n----------------------------------------Seg-Grad-Cam----------------------------------------------------\n')

        if self.cuda:
            input_tensor = input_tensor.cuda()
        if self.compute_input_gradient:
            input_tensor = torch.autograd.Variable(input_tensor, requires_grad=True)

        outputs = self.activations_and_grads(input_tensor)
        if targets is None:
            target_categories = np.argmax(outputs.cpu().data.numpy(), axis=-1)
            targets = [ClassifierOutputTarget(category) for category in target_categories]

        if self.uses_gradients:
            self.model.zero_grad()
            loss = sum([target(output) for target, output in zip(targets, outputs)])
            loss.backward(retain_graph=True)

        #----remove------

        input_grad = input_tensor.grad.data.cpu().numpy()
        grads_list = [g.cpu().data.numpy() for g in self.activations_and_grads.gradients]
        cam_per_target_layer = []
        target_size = input_tensor.size(-1), input_tensor.size(-2)

        gradient_multiplied_input = input_grad * input_tensor.data.cpu().numpy()
        gradient_multiplied_input = np.abs(gradient_multiplied_input)
        gradient_multiplied_input = scale_accross_batch_and_channels(
            gradient_multiplied_input,
            target_size)
        cam_per_target_layer.append(gradient_multiplied_input)

        # Loop over the saliency image from every layer
        assert(len(self.bias_data) == len(grads_list))
        for bias, grads in zip(self.bias_data, grads_list):
            bias = bias[None, :, None, None]
            # In the paper they take the absolute value,
            # but possibily taking only the positive gradients will work
            # better.
            bias_grad = np.abs(bias * grads)
            result = scale_accross_batch_and_channels(
                bias_grad, target_size)
            result = np.sum(result, axis=1)
            cam_per_target_layer.append(result[:, None, :])
        cam_per_target_layer = np.concatenate(cam_per_target_layer, axis=1)
        if eigen_smooth:
            # Resize to a smaller image, since this method typically has a very large number of channels,
            # and then consumes a lot of memory
            cam_per_target_layer = scale_accross_batch_and_channels(
                cam_per_target_layer, (target_size[0] // 8, target_size[1] // 8))
            cam_per_target_layer = get_2d_projection(cam_per_target_layer)
            cam_per_target_layer = cam_per_target_layer[:, None, :, :]
            cam_per_target_layer = scale_accross_batch_and_channels(
                cam_per_target_layer,
                target_size)
        else:
            cam_per_target_layer = np.sum(
                cam_per_target_layer, axis=1)[:, None, :]


        result = np.sum(cam_per_target_layer, axis=1)
        return scale_cam_image(result) # result

    def __call__(self, input_tensor: torch.Tensor, targets: List[torch.nn.Module] = None, aug_smooth: bool = False,
                 eigen_smooth: bool = False) -> np.ndarray:
        # Smooth the CAM result with test time augmentation
        if aug_smooth is True:
            transforms = tta.Compose(
                [
                    tta.HorizontalFlip(),
                    tta.Multiply(factors=[0.9, 1, 1.1]),
                ]
            )
            cams = []
            for transform in transforms:
                augmented_tensor = transform.augment_image(input_tensor)
                cam = self.forward(augmented_tensor, targets, eigen_smooth)
                # The ttach library expects a tensor of size BxCxHxW
                cam = cam[:, None, :, :]
                cam = torch.from_numpy(cam)
                cam = transform.deaugment_mask(cam)
                # Back to numpy float32, HxW
                cam = cam.numpy()
                cam = cam[:, 0, :, :]
                cams.append(cam)
            cam = np.mean(np.float32(cams), axis=0)
            return cam
        else:
            return self.forward(input_tensor, targets, eigen_smooth)

    def __del__(self):
        self.activations_and_grads.release()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.activations_and_grads.release()
        if isinstance(exc_value, IndexError):
            # Handle IndexError here...
            print(
                f"An exception occurred in CAM with block: {exc_type}. Message: {exc_value}")
            return True

In [10]:
# Apply the adapted CAM-based Extensions on the considered dataset

class SemanticSegmentationTarget:
    def __init__(self, category, mask):
        self.category = category
        self.mask = torch.from_numpy(mask)
        if torch.cuda.is_available():
            self.mask = self.mask.cuda()

    def __call__(self, model_output):
        return (model_output[self.category, :, : ] * self.mask).sum()


def defaultScales():
    classes_cmap = plt.get_cmap('Spectral', 20)
    scale_fig = 2
    fonts = 15
    scatter_size = 330 * scale_fig
    return classes_cmap, scale_fig, fonts, scatter_size


def show_cam_image_whu(full_img_rgba, grayscale_cam_EX, image_weight):
    heatmap_sgc = cv2.applyColorMap(np.uint8(255 * grayscale_cam_EX), cv2.COLORMAP_JET)
    heatmap_sgc = cv2.cvtColor(heatmap_sgc, cv2.COLOR_BGR2RGB)
    heatmap_sgc = np.float32(heatmap_sgc) / 255
    if full_img.shape[-1]==4:
        heatmap_sgc = cv2.cvtColor(heatmap_sgc,cv2.COLOR_RGB2RGBA)
    Exmap_sgc = (1 - image_weight) * heatmap_sgc + image_weight * full_img_rgba
    Exmap_sgc = Exmap_sgc / np.max(Exmap_sgc)
    Exmap_sgc = np.uint8(255 * Exmap_sgc)

    return Exmap_sgc, heatmap_sgc


def prob_2_entropy(prob):
    """ convert probabilistic prediction maps to weighted self-information maps
    """
    n, c, h, w = prob.size()
    return -torch.mul(prob, torch.log2(prob + 1e-30)) / np.log2(c)


def XAI_EVAL(E_sgc, full_img_gt, model, rrp_info,target_category):

    x_sgc = totensor(E_sgc)
    # x_sgc = x_sgc.cuda()
    with torch.no_grad():
        y_pred_sgc = model(x_sgc)
        y_pred_sgc = unpad_resize(y_pred_sgc,rrp_info)

    mask_tensor_sgc = y_pred_sgc[0,...]
    mask_sgc = y_pred_sgc[0,...].cpu().numpy().transpose(1,2,0)
    target_mask_f = np.float32(mask_sgc[:,:,target_category]) * full_img_gt
    target_Confidence_score = target_mask_f[np.nonzero(target_mask_f)]
    target_Confidence_score_sgc = np.mean(target_Confidence_score)

    logist_softmax_entropy_sgc = prob_2_entropy(y_pred_sgc)
    target_entropy_mask_sgc = logist_softmax_entropy_sgc[0, :, :, :].detach().cpu().numpy().transpose(1,2,0)
    target_entropy_mask_class_sgc = target_entropy_mask_sgc[:,:,target_category] * full_img_gt
    target_entropy_mask_class_sgc_sc = (np.mean(target_entropy_mask_class_sgc))
    return target_Confidence_score_sgc, target_entropy_mask_class_sgc_sc


images_dir = r'./dataset/images'
gt_dir = r'./dataset/gt'





ticks = np.linspace(0, 1, 6, endpoint=True)
classes_cmap, scale_fig, fonts, scatter_size = defaultScales()




decoder_idx = 1
thres = 0.4
n_xai = 6
n_imgs = 1
target_layers =  [model.decoder.blocks[decoder_idx - 1]]
target_category = 0
XAI_method = ["grad_cam", "hires_cam", "ew_cam", "grad_cam_pp", "x_grad_cam","score_cam", "layer_cam", "eigen_cam", "eigen_grad_cam"]

image_weight = 0.006

directory_images = os.fsencode(images_dir)
directory_gt = os.fsencode(gt_dir)


number_testing_images = 0

Model_Seg_Score = np.zeros((n_imgs))
Model_Seg_Entropy = np.zeros((n_imgs))

Seg_Score_M1 = np.zeros((n_xai, n_imgs))
Seg_Entropy_M1 = np.zeros((n_xai, n_imgs))

Seg_Score_M2 = np.zeros((n_xai, n_imgs))
Seg_Entropy_M2 = np.zeros((n_xai, n_imgs))

Seg_Score_M3 = np.zeros((n_xai, n_imgs))
Seg_Entropy_M3 = np.zeros((n_xai, n_imgs))

counter = 0


for (file_img, file_gt) in zip(os.listdir(directory_images), os.listdir(directory_gt) ):

    print(file_img, file_gt)
    if file_img == file_gt:
        number_testing_images = number_testing_images + 1
        print("Testing Image: ", number_testing_images)
        filename = os.fsdecode(file_img)
        filename_gt = os.fsdecode(file_gt)
        if filename.endswith(".png") and filename_gt.endswith(".png"):
            raster_file = rio.open(f'{images_dir}/{filename}')
            full_img = raster_file.read().transpose(1,2,0)
            full_img,rrp_info = ratio_resize_pad(full_img, ratio = None)
            full_img_rgba = full_img
            print("Input Image Shape: ", full_img.shape)
            if full_img.shape[-1]==4:full_img = cv2.cvtColor(full_img,cv2.COLOR_RGBA2RGB) #WHU images are RGBA
            # read gt mask
            raster_file_gt = rio.open(f'{gt_dir}/{filename_gt}')
            full_img_gt = raster_file_gt.read().transpose(1,2,0)
            full_img_gt,rrp_info_gt = ratio_resize_pad(full_img_gt, ratio = None)
            full_img_gt = np.float32(full_img_gt) / np.max(full_img_gt)
            print("GT mask shape: ", full_img_gt.shape)


            full_img = normalize(full_img)
            x = totensor(full_img)
            # x = x.cuda()

            with torch.no_grad():
                y_pred = model(x)
                y_pred = unpad_resize(y_pred,rrp_info)



            mask_tensor = y_pred[0,...]
            print('predicated tensor shape: ', mask_tensor.shape)
            mask = y_pred[0,...].cpu().numpy().transpose(1,2,0)

            target_mask_float = np.float32(mask[:,:,target_category]) * full_img_gt
            target_Confidence_score = target_mask_float[np.nonzero(target_mask_float)]
            target_Confidence_score = np.mean(target_Confidence_score)
            print("Model Target Confidence Score: ", target_Confidence_score)

            logist_softmax_entropy = prob_2_entropy(y_pred)
            target_entropy_mask = logist_softmax_entropy[0, :, :, :].detach().cpu().numpy().transpose(1,2,0)
            target_entropy_mask_class = target_entropy_mask[:,:,target_category] * full_img_gt
            model_entropy = np.mean(target_entropy_mask_class)
            print("Model Target Entropy Score: ", model_entropy )


            targets = [SemanticSegmentationTarget(target_category, target_mask_float)]

            nan_condition = np.count_nonzero(target_mask_float)
            if nan_condition != 0:

                print('running seg-grad-cam...')
                with GRADCAM_Extensions(extension = XAI_method[0], model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as cam:
                    grayscale_cam_EX = cam(input_tensor=x, targets=targets)[0, :]
                    [Exmap_sgc, heatmap_sgc_rgba] = show_cam_image_whu(full_img_rgba, grayscale_cam_EX, image_weight)

                    # Evaluation
                    im_bw_sgc = cv2.threshold(grayscale_cam_EX, thres, 1, cv2.THRESH_BINARY)[1]
                    # M1: Background Only
                    E_sgc_M1 = full_img * np.logical_not(im_bw_sgc)[..., None]
                    [confidence_sgc_M1, entropy_sgc_M1] = XAI_EVAL(E_sgc_M1, full_img_gt, model, rrp_info,target_category)
                    print("M1 -- SGC Target Confidence Score: ", confidence_sgc_M1," Entropy Score: ", entropy_sgc_M1)

                    # M2: Highlighted Only
                    E_sgc_M2 = full_img * im_bw_sgc[..., None]
                    [confidence_sgc_M2, entropy_sgc_M2] = XAI_EVAL(E_sgc_M2, full_img_gt, model, rrp_info,target_category)
                    print("M2 -- SGC Target Confidence Score: ", confidence_sgc_M2," Entropy Score: ", entropy_sgc_M2)

                    # M3 : Highlighted + GT
                    union_gt_sgc = np.ma.mask_or(full_img_gt,im_bw_sgc)
                    E_sgc_M3 = full_img * union_gt_sgc[..., None]

                    [confidence_sgc_M3, entropy_sgc_M3] = XAI_EVAL(E_sgc_M3, full_img_gt, model, rrp_info,target_category)
                    print("M3 -- SGC Target Confidence Score: ", confidence_sgc_M3," Entropy Score: ", entropy_sgc_M3)
                #______________________________
                print('running GradCam++ ...')
                with GRADCAM_Extensions(extension = XAI_method[3], model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as campp:
                    grayscale_cam_EX_Plusplus = campp(input_tensor=x, targets=targets)[0, :]
                    [Exmap_sgcpp, heatmap_sgcpp_rgba] = show_cam_image_whu(full_img_rgba, grayscale_cam_EX_Plusplus, image_weight)

                		# Evaluation
                    im_bw_sgcpp = cv2.threshold(grayscale_cam_EX_Plusplus, thres, 1, cv2.THRESH_BINARY)[1]

                    # M1: Background Only
                    E_sgcpp_M1 = full_img * np.logical_not(im_bw_sgcpp)[..., None]
                    [confidence_sgcpp_M1, entropy_sgcpp_M1] = XAI_EVAL(E_sgcpp_M1, full_img_gt, model, rrp_info,target_category)
                    print("M1 -- SGC++ Target Confidence Score: ", confidence_sgcpp_M1," Entropy Score: ", entropy_sgcpp_M1)

										# M2: Highlighted Only
                    E_sgcpp_M2 = full_img * im_bw_sgcpp[..., None]
                    [confidence_sgcpp_M2, entropy_sgcpp_M2] = XAI_EVAL(E_sgcpp_M2, full_img_gt, model, rrp_info,target_category)
                    print("M2 -- SGC++ Target Confidence Score: ", confidence_sgcpp_M2," Entropy Score: ", entropy_sgcpp_M2)

              			# M3: Highlighted + GT
                    union_gt_sgcpp = np.ma.mask_or(full_img_gt,im_bw_sgcpp)
                    E_sgcpp_M3 = full_img * union_gt_sgcpp[..., None]
                    [confidence_sgcpp_M3, entropy_sgcpp_M3] = XAI_EVAL(E_sgcpp_M3, full_img_gt, model, rrp_info,target_category)
                    print("M3 -- SGC++ Target Confidence Score: ", confidence_sgcpp_M3," Entropy Score: ", entropy_sgcpp_M3)

								#___________________________
                print('running XGradCam...')
                with GRADCAM_Extensions(extension = XAI_method[4], model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as xcam:
                    grayscale_cam_EX_X = xcam(input_tensor=x, targets=targets)[0, :]
                    [Exmap_Xsgc, heatmap_Xsgc_rgba] = show_cam_image_whu(full_img_rgba, grayscale_cam_EX_X, image_weight)
										# Evaluation
                    im_bw_Xsgc = cv2.threshold(grayscale_cam_EX_X, thres, 1, cv2.THRESH_BINARY)[1]

                    # M1: Background Only
                    E_Xsgc_M1 = full_img * np.logical_not(im_bw_Xsgc)[..., None]
                    [confidence_Xsgc_M1, entropy_Xsgc_M1] = XAI_EVAL(E_Xsgc_M1, full_img_gt, model, rrp_info,target_category)
                    print("M1 -- XSGC Target Confidence Score: ", confidence_Xsgc_M1," Entropy Score: ", entropy_Xsgc_M1)

										# M2: Highlighted Only
                    E_Xsgc_M2 = full_img * im_bw_Xsgc[..., None]
                    [confidence_Xsgc_M2, entropy_Xsgc_M2] = XAI_EVAL(E_Xsgc_M2, full_img_gt, model, rrp_info,target_category)
                    print("M2 -- XSGC Target Confidence Score: ", confidence_Xsgc_M2," Entropy Score: ", entropy_Xsgc_M2)

              			# M3: Highlighted + GT
                    union_gt_Xsgc = np.ma.mask_or(full_img_gt,im_bw_Xsgc)
                    E_Xsgc_M3 = full_img * union_gt_Xsgc[..., None]
                    [confidence_Xsgc_M3, entropy_Xsgc_M3] = XAI_EVAL(E_Xsgc_M3, full_img_gt, model, rrp_info,target_category)
                    print("M3 -- XSGC Target Confidence Score: ", confidence_Xsgc_M3," Entropy Score: ", entropy_Xsgc_M3)

								#___________________________
                print('running ScoreCam...')
                with GRADCAM_Extensions(extension = XAI_method[5], model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as scorecam:
                    grayscale_cam_EX_Score = scorecam(input_tensor=x, targets=targets)[0, :]
                    [Exmap_scoresgc, heatmap_scoresgc_rgba] = show_cam_image_whu(full_img_rgba, grayscale_cam_EX_Score, image_weight)

                    # Evaluation
                    im_bw_ssgc = cv2.threshold(grayscale_cam_EX_Score, thres, 1, cv2.THRESH_BINARY)[1]

                    # M1: Background Only
                    E_ssgc_M1 = full_img * np.logical_not(im_bw_ssgc)[..., None]
                    [confidence_ssgc_M1, entropy_ssgc_M1] = XAI_EVAL(E_ssgc_M1, full_img_gt, model, rrp_info,target_category)
                    print("M1 -- Score SGC Target Confidence Score: ", confidence_ssgc_M1," Entropy Score: ", entropy_ssgc_M1)

										# M2: Highlighted Only
                    E_ssgc_M2 = full_img * im_bw_ssgc[..., None]
                    [confidence_ssgc_M2, entropy_ssgc_M2] = XAI_EVAL(E_ssgc_M2, full_img_gt, model, rrp_info,target_category)
                    print("M2 -- Score SGC Target Confidence Score: ", confidence_ssgc_M2," Entropy Score: ", entropy_ssgc_M2)

              			# M3: Highlighted + GT
                    union_gt_ssgc = np.ma.mask_or(full_img_gt,im_bw_ssgc)
                    E_ssgc_M3 = full_img * union_gt_ssgc[..., None]
                    [confidence_ssgc_M3, entropy_ssgc_M3] = XAI_EVAL(E_ssgc_M3, full_img_gt, model, rrp_info,target_category)
                    print("M3 -- Score SGC Target Confidence Score: ", confidence_ssgc_M3," Entropy Score: ", entropy_ssgc_M3)

								#__________________________
                print('running EigenCam...')
                with GRADCAM_Extensions(extension = XAI_method[7], model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as ecam:
                    grayscale_cam_EX_eigen = ecam(input_tensor=x, targets=targets)[0, :]
                    [Exmap_eigensgc, heatmap_eigensgc_rgba] = show_cam_image_whu(full_img_rgba, grayscale_cam_EX_eigen, image_weight)

										# Evaluation
                    im_bw_esgc = cv2.threshold(grayscale_cam_EX_eigen, thres, 1, cv2.THRESH_BINARY)[1]

                    # M1: Background Only
                    E_esgc_M1 = full_img * np.logical_not(im_bw_esgc)[..., None]
                    [confidence_esgc_M1, entropy_esgc_M1] = XAI_EVAL(E_esgc_M1, full_img_gt, model, rrp_info,target_category)
                    print("M1 -- Eigen SGC Target Confidence Score: ", confidence_esgc_M1," Entropy Score: ", entropy_esgc_M1)

										# M2: Highlighted Only
                    E_esgc_M2 = full_img * im_bw_esgc[..., None]
                    [confidence_esgc_M2, entropy_esgc_M2] = XAI_EVAL(E_esgc_M2, full_img_gt, model, rrp_info,target_category)
                    print("M2 -- Eigen SGC Target Confidence Score: ", confidence_esgc_M2," Entropy Score: ", entropy_esgc_M2)

              			# M3: Highlighted + GT
                    union_gt_esgc = np.ma.mask_or(full_img_gt,im_bw_esgc)
                    E_esgc_M3 = full_img * union_gt_esgc[..., None]
                    [confidence_esgc_M3, entropy_esgc_M3] = XAI_EVAL(E_esgc_M3, full_img_gt, model, rrp_info,target_category)
                    print("M3 -- Eigen SGC Target Confidence Score: ", confidence_esgc_M3," Entropy Score: ", entropy_esgc_M3)

                Model_Seg_Score[counter] = target_Confidence_score
                Model_Seg_Entropy[counter] = model_entropy

                Seg_Score_M1[0,counter] = confidence_sgc_M1
                Seg_Score_M1[1,counter] = confidence_sgcpp_M1
                Seg_Score_M1[2,counter] = confidence_Xsgc_M1
                Seg_Score_M1[3,counter] = confidence_ssgc_M1
                Seg_Score_M1[4,counter] = confidence_esgc_M1

                Seg_Entropy_M1[0,counter] = entropy_sgc_M1
                Seg_Entropy_M1[1,counter] = entropy_sgcpp_M1
                Seg_Entropy_M1[2,counter] = entropy_Xsgc_M1
                Seg_Entropy_M1[3,counter] = entropy_ssgc_M1
                Seg_Entropy_M1[4,counter] = entropy_esgc_M1


                Seg_Score_M2[0,counter] = confidence_sgc_M2
                Seg_Score_M2[1,counter] = confidence_sgcpp_M2
                Seg_Score_M2[2,counter] = confidence_Xsgc_M2
                Seg_Score_M2[3,counter] = confidence_ssgc_M2
                Seg_Score_M2[4,counter] = confidence_esgc_M2

                Seg_Entropy_M2[0,counter] = entropy_sgc_M2
                Seg_Entropy_M2[1,counter] = entropy_sgcpp_M2
                Seg_Entropy_M2[2,counter] = entropy_Xsgc_M2
                Seg_Entropy_M2[3,counter] = entropy_ssgc_M2
                Seg_Entropy_M2[4,counter] = entropy_esgc_M2

                Seg_Score_M3[0,counter] = confidence_sgc_M3
                Seg_Score_M3[1,counter] = confidence_sgcpp_M3
                Seg_Score_M3[2,counter] = confidence_Xsgc_M3
                Seg_Score_M3[3,counter] = confidence_ssgc_M3
                Seg_Score_M3[4,counter] = confidence_esgc_M3

                Seg_Entropy_M3[0,counter] = entropy_sgc_M3
                Seg_Entropy_M3[1,counter] = entropy_sgcpp_M3
                Seg_Entropy_M3[2,counter] = entropy_Xsgc_M3
                Seg_Entropy_M3[3,counter] = entropy_ssgc_M3
                Seg_Entropy_M3[4,counter] = entropy_esgc_M3


                counter = counter + 1

                if(number_testing_images == 2):
                    break



                fig = plt.figure(figsize=(8 * scale_fig, 8 * scale_fig))

                plt.subplot(3,3,1)
                plt.axis('off')
                plt.imshow(full_img_rgba)
                plt.title('Input image', fontsize=fonts)

                plt.subplot(3,3,2)
                plt.axis('off')
                plt.imshow(full_img_gt, cmap='gray', vmin=0, vmax=1)
                plt.title('Ground Truth (GT) Mask', fontsize=fonts)

                plt.subplot(3,3,3)
                plt.axis('off')
                plt.imshow(target_mask_float,  cmap='gray', vmin=0, vmax=1)
                plt.title('Predicted Mask', fontsize=fonts)

                plt.subplot(3,3,4)
                plt.axis('off')
                plt.imshow(Exmap_sgc)
                plt.title('Seg-Grad-CAM', fontsize=fonts)

                plt.subplot(3,3,5)
                plt.axis('off')
                plt.imshow(Exmap_sgcpp)
                plt.title('Seg-Grad-CAM ++', fontsize=fonts)

                plt.subplot(3,3,6)
                plt.axis('off')
                plt.imshow(Exmap_Xsgc)
                plt.title('Seg-XGrad-CAM', fontsize=fonts)

                plt.subplot(3,3,7)
                plt.axis('off')
                plt.imshow(Exmap_scoresgc)
                plt.title('Seg-Score-CAM', fontsize=fonts)

                plt.subplot(3,3,8)
                plt.axis('off')
                plt.imshow(Exmap_eigensgc)
                plt.title('Seg-Eigen-CAM', fontsize=fonts)


                # plt.subplot(3,3,9)
                # plt.axis('off')
                # plt.imshow(Exmap_ablsgc)
                # plt.title('Seg-Ablation-CAM', fontsize=fonts)

                # Save the full figure...
                fig.savefig('./results/SegGradCam_Extensions_{}_{}.png'.format(filename,decoder_idx), bbox_inches='tight', pad_inches=0)



                fig = plt.figure(figsize=(8 * scale_fig, 8 * scale_fig))

                plt.subplot(3,3,1)
                plt.axis('off')
                plt.imshow(full_img_rgba)
                plt.title('Input image', fontsize=fonts)

                plt.subplot(3,3,2)
                plt.axis('off')
                plt.imshow(full_img_gt, cmap='gray', vmin=0, vmax=1)
                plt.title('Ground Truth (GT) Mask', fontsize=fonts)

                plt.subplot(3,3,3)
                plt.axis('off')
                plt.imshow(target_mask_float,  cmap='gray', vmin=0, vmax=1)
                plt.title('Predicted Mask', fontsize=fonts)

                plt.subplot(3,3,4)
                plt.axis('off')
                plt.imshow(E_sgc_M3)
                plt.title('Seg-Grad-CAM', fontsize=fonts)

                plt.subplot(3,3,5)
                plt.axis('off')
                plt.imshow(E_sgcpp_M3)
                plt.title('Seg-Grad-CAM ++', fontsize=fonts)

                plt.subplot(3,3,6)
                plt.axis('off')
                plt.imshow(E_Xsgc_M3)
                plt.title('Seg-XGrad-CAM', fontsize=fonts)

                plt.subplot(3,3,7)
                plt.axis('off')
                plt.imshow(E_ssgc_M3)
                plt.title('Seg-Score-CAM', fontsize=fonts)

                plt.subplot(3,3,8)
                plt.axis('off')
                plt.imshow(E_esgc_M3)
                plt.title('Seg-Eigen-CAM', fontsize=fonts)


                # plt.subplot(3,3,9)
                # plt.axis('off')
                # plt.imshow(Exmap_ablsgc)
                # plt.title('Seg-Ablation-CAM', fontsize=fonts)

                # Save the full figure...
                fig.savefig('./results/SegGradCam_Modified_M3_{}_{}.png'.format(filename,decoder_idx), bbox_inches='tight', pad_inches=0)





Output hidden; open in https://colab.research.google.com to view.

# Task 1:Evaluating XAI Methods based on M1, M2, M3 across different thresholds

In [11]:
XAI_method_t1 = ["grad_cam", "grad_cam_pp", "x_grad_cam", "score_cam", "eigen_cam"]
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5]

In [12]:
results_t1 = {
    method: {
        th: {'M1': {}, 'M2': {}, 'M3': {}} for th in thresholds
    }
    for method in XAI_method_t1
}

for method_name in XAI_method_t1:
    print(f'Running {method_name} ...')

    with GRADCAM_Extensions(extension=method_name, model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as cam:
        grayscale_cam = cam(input_tensor=x, targets=targets)[0, :]

        for thres in thresholds:

            im_bw = cv2.threshold(grayscale_cam, thres, 1, cv2.THRESH_BINARY)[1]

            # M1: Background Only
            E_M1 = full_img * np.logical_not(im_bw)[..., None]
            confidence_M1, entropy_M1 = XAI_EVAL(E_M1, full_img_gt, model, rrp_info, target_category)

            # M2: Highlighted Only
            E_M2 = full_img * im_bw[..., None]
            confidence_M2, entropy_M2 = XAI_EVAL(E_M2, full_img_gt, model, rrp_info, target_category)

            # M3: Highlighted + GT
            union_mask = np.ma.mask_or(full_img_gt, im_bw)
            E_M3 = full_img * union_mask[..., None]
            confidence_M3, entropy_M3 = XAI_EVAL(E_M3, full_img_gt, model, rrp_info, target_category)

            results_t1[method_name][thres]['M1'] = {'confidence': confidence_M1, 'entropy': entropy_M1}
            results_t1[method_name][thres]['M2'] = {'confidence': confidence_M2, 'entropy': entropy_M2}
            results_t1[method_name][thres]['M3'] = {'confidence': confidence_M3, 'entropy': entropy_M3}


Running grad_cam ...
Running grad_cam_pp ...
Running x_grad_cam ...
Running score_cam ...


100%|██████████| 32/32 [10:18<00:00, 19.32s/it]


Running eigen_cam ...


**Convert results into dataframe to print**

In [13]:
rows = []
for method in results_t1:
    for th in results_t1[method]:
        for metric in results_t1[method][th]:
            row = {
                'Method': method,
                'Threshold': th,
                'Metric': metric,
                'Confidence': results_t1[method][th][metric]['confidence'],
                'Entropy': results_t1[method][th][metric]['entropy']
            }
            rows.append(row)

df_results_t1 = pd.DataFrame(rows)

df_results_t1 = df_results_t1.sort_values(by=['Method', 'Threshold', 'Metric'])

print(df_results_t1.to_string(index=False))

     Method  Threshold Metric  Confidence  Entropy
  eigen_cam        0.1     M1    0.911135 0.001452
  eigen_cam        0.1     M2    0.207058 0.003010
  eigen_cam        0.1     M3    0.147083 0.002163
  eigen_cam        0.2     M1    0.911754 0.001386
  eigen_cam        0.2     M2    0.575863 0.005440
  eigen_cam        0.2     M3    0.059528 0.001578
  eigen_cam        0.3     M1    0.910741 0.001438
  eigen_cam        0.3     M2    0.596144 0.005520
  eigen_cam        0.3     M3    0.054794 0.001619
  eigen_cam        0.4     M1    0.911137 0.001412
  eigen_cam        0.4     M2    0.656418 0.005555
  eigen_cam        0.4     M3    0.042779 0.001307
  eigen_cam        0.5     M1    0.911723 0.001389
  eigen_cam        0.5     M2    0.731848 0.005596
  eigen_cam        0.5     M3    0.035465 0.001225
   grad_cam        0.1     M1    0.878462 0.002150
   grad_cam        0.1     M2    0.161230 0.000952
   grad_cam        0.1     M3    0.357960 0.002641
   grad_cam        0.2     M1  

**Best XAI Method for each Threshold for each Evaluation Metric**

In [14]:
best_methods = []

for (thresh, metric), group in df_results_t1.groupby(['Threshold', 'Metric']):
    if metric == 'M1':
        # Lowest confidence
        best_row = group.loc[group['Confidence'].idxmin()]
    elif metric == 'M2':
        # Highest confidence
        best_row = group.loc[group['Confidence'].idxmax()]
    elif metric == 'M3':
        # Lowest entropy
        best_row = group.loc[group['Entropy'].idxmin()]
    else:
        continue

    best_methods.append(best_row)

best_methods_df = pd.DataFrame(best_methods)

best_methods_df = best_methods_df.sort_values(['Metric', 'Threshold']).reset_index(drop=True)
print(best_methods_df[['Threshold', 'Metric', 'Method', 'Confidence', 'Entropy']])


    Threshold Metric       Method  Confidence   Entropy
0         0.1     M1  grad_cam_pp    0.090199  0.001079
1         0.2     M1    score_cam    0.032427  0.000289
2         0.3     M1    score_cam    0.039928  0.000661
3         0.4     M1    score_cam    0.197047  0.001647
4         0.5     M1    score_cam    0.535484  0.002970
5         0.1     M2    score_cam    0.904449  0.001422
6         0.2     M2    score_cam    0.898507  0.001354
7         0.3     M2    score_cam    0.869302  0.001446
8         0.4     M2     grad_cam    0.725109  0.002610
9         0.5     M2     grad_cam    0.768169  0.002974
10        0.1     M3  grad_cam_pp    0.901705  0.001388
11        0.2     M3    score_cam    0.905012  0.001380
12        0.3     M3     grad_cam    0.035126  0.001178
13        0.4     M3   x_grad_cam    0.033650  0.001117
14        0.5     M3  grad_cam_pp    0.033526  0.001117


* **Lowest M1 Confidence:** Background alone should give low confidence (excluded irrelevant info)
* **Highest M2 Confidence:** Highlighted region alone should give high confidence (important info captured)
* **Lowest M3 Entropy:** Explanation aligns well with GT and model is confident

**Best Overall XAI Method is Most Frequent Best**

In [15]:
method_score = best_methods_df['Method'].value_counts()

print("Method Selection Frequency:")
print(method_score)

best_overall_method = method_score.idxmax()
print(f"\nBest overall XAI method: {best_overall_method}")


Method Selection Frequency:
Method
score_cam      8
grad_cam_pp    3
grad_cam       3
x_grad_cam     1
Name: count, dtype: int64

Best overall XAI method: score_cam


**Best Threshold Value(s)**

In [17]:
def normalize(arr):
    return (arr - np.min(arr)) / (np.max(arr) - np.min(arr))

best_thresholds = {}

for method in df_results_t1['Method'].unique():
    df_m = df_results_t1[df_results_t1['Method'] == method]

    conf_pivot = df_m.pivot(index='Threshold', columns='Metric', values='Confidence')
    entropy_pivot = df_m.pivot(index='Threshold', columns='Metric', values='Entropy')

    m1_conf_norm = normalize(conf_pivot['M1'].values)
    m2_conf_norm = normalize(conf_pivot['M2'].values)
    m3_entropy_norm = normalize(entropy_pivot['M3'].values)

    # Combined score: low M1 conf, high M2 conf, low M3 entropy
    combined_score = m1_conf_norm + (1 - m2_conf_norm) + m3_entropy_norm

    best_idx = np.argmin(combined_score)
    best_thresh = conf_pivot.index[best_idx]

    best_thresholds[method] = {
        'best_threshold': best_thresh,
        'combined_scores': dict(zip(conf_pivot.index, combined_score))
    }

for method, info in best_thresholds.items():
    print(f"Method: {method}")
    print(f" Best threshold: {info['best_threshold']}")
    print()


Method: eigen_cam
 Best threshold: 0.4

Method: grad_cam
 Best threshold: 0.3

Method: grad_cam_pp
 Best threshold: 0.1

Method: score_cam
 Best threshold: 0.2

Method: x_grad_cam
 Best threshold: 0.5



---
The heatmap threshold strongly influences the faithfulness and clarity of XAI explanations by controlling which pixels are considered important.

# Task 2: Drop in Segmentation Score Metric After Masking Important Regions

IoU measures intersection over union of model's prediction on image and ground-truth mask

Drop in IoU measures how much the model performs when we mask the img using xai to show only the parts it thinks are imp.

In [18]:
thres = 0.4

In [19]:
def XAI_EVAL_Drop(E_sgc, full_img_gt, model, full_img, rrp_info, target_category):
    import torch.nn.functional as F
    import numpy as np

    def compute_iou(pred_mask, gt_mask):
        intersection = np.logical_and(pred_mask, gt_mask).sum()
        union = np.logical_or(pred_mask, gt_mask).sum()
        if union == 0: return 0.0
        return intersection / union

    x_sgc = totensor(E_sgc)
    with torch.no_grad():
        # Prediction from masked input (explanation-applied)
        y_pred_sgc = model(x_sgc)
        y_pred_sgc = unpad_resize(y_pred_sgc, rrp_info)

        # Prediction of full img
        x_full = totensor(full_img)
        y_pred_full = model(x_full)
        y_pred_full = unpad_resize(y_pred_full, rrp_info)

    pred_sgc = torch.argmax(y_pred_sgc[0], dim=0).cpu().numpy()
    pred_full = torch.argmax(y_pred_full[0], dim=0).cpu().numpy()
    gt_mask = full_img_gt.astype(np.uint8)

    iou_full = compute_iou(pred_full == target_category, gt_mask == target_category)
    iou_sgc = compute_iou(pred_sgc == target_category, gt_mask == target_category)

    drop_in_iou = iou_full - iou_sgc

    return drop_in_iou


In [20]:
drop_results = {method: {} for method in XAI_method_t1}

for method_name in XAI_method_t1:
    print(f'running {method_name} ...')

    with GRADCAM_Extensions(extension=method_name, model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as cam:
        grayscale_cam = cam(input_tensor=x, targets=targets)[0, :]

        im_bw = cv2.threshold(grayscale_cam, thres, 1, cv2.THRESH_BINARY)[1]

        # M3: Highlighted + GT
        union_mask = np.ma.mask_or(full_img_gt, im_bw)
        E_img = full_img * union_mask[..., None]
        drop_result = XAI_EVAL_Drop(E_img, full_img_gt, model, full_img, rrp_info, target_category)

        print(f"{method_name} Drop in Segmentation: ", drop_result)
        drop_results[method_name] = {
            'drop': drop_result
        }


running grad_cam ...
grad_cam Drop in Segmentation:  -0.23857735226199223
running grad_cam_pp ...
grad_cam_pp Drop in Segmentation:  -0.24065775090664665
running x_grad_cam ...
x_grad_cam Drop in Segmentation:  -0.23857641220598186
running score_cam ...


100%|██████████| 32/32 [10:18<00:00, 19.34s/it]


score_cam Drop in Segmentation:  -0.30579284561942155
running eigen_cam ...
eigen_cam Drop in Segmentation:  0.027292047602201046


In [21]:
ranked_methods_t2 = sorted(drop_results.items(), key=lambda x: x[1]['drop'])

print("XAI Methods Ranked by Faithfulness to the model’s true reasoning:")
for i, (method, result) in enumerate(ranked_methods_t2, start=1):
    print(f"{i}. {method}: Drop = {result['drop']:.4f}")


XAI Methods Ranked by Faithfulness to the model’s true reasoning:
1. score_cam: Drop = -0.3058
2. grad_cam_pp: Drop = -0.2407
3. grad_cam: Drop = -0.2386
4. x_grad_cam: Drop = -0.2386
5. eigen_cam: Drop = 0.0273


In [23]:
best_method_t2, best_result_t2 = ranked_methods_t2[0]

print(f" Best XAI method (lowest drop in IoU): {best_method_t2} with drop = {best_result_t2['drop']:.4f}")


 Best XAI method (lowest drop in IoU): score_cam with drop = -0.3058


---
When applying the M3 evaluation, a lower IoU drop indicates that the explanation method successfully highlighted the regions most critical to the model’s segmentation of the target class, even beyond the ground truth boundaries—demonstrating higher faithfulness to the model’s reasoning

# Task 3: Using 2nd Decoder

In [24]:
class SemanticSegmentationTarget:
    def __init__(self, category, mask):
        self.category = category
        self.mask = torch.from_numpy(mask)
        if torch.cuda.is_available():
            self.mask = self.mask.cuda()

    def __call__(self, model_output):
        return (model_output[self.category, :, : ] * self.mask).sum()


def defaultScales():
    classes_cmap = plt.get_cmap('Spectral', 20)
    scale_fig = 2
    fonts = 15
    scatter_size = 330 * scale_fig
    return classes_cmap, scale_fig, fonts, scatter_size


def show_cam_image_whu(full_img_rgba, grayscale_cam_EX, image_weight):
    heatmap_sgc = cv2.applyColorMap(np.uint8(255 * grayscale_cam_EX), cv2.COLORMAP_JET)
    heatmap_sgc = cv2.cvtColor(heatmap_sgc, cv2.COLOR_BGR2RGB)
    heatmap_sgc = np.float32(heatmap_sgc) / 255
    if full_img.shape[-1]==4:
        heatmap_sgc = cv2.cvtColor(heatmap_sgc,cv2.COLOR_RGB2RGBA)
    Exmap_sgc = (1 - image_weight) * heatmap_sgc + image_weight * full_img_rgba
    Exmap_sgc = Exmap_sgc / np.max(Exmap_sgc)
    Exmap_sgc = np.uint8(255 * Exmap_sgc)

    return Exmap_sgc, heatmap_sgc


def prob_2_entropy(prob):
    """ convert probabilistic prediction maps to weighted self-information maps
    """
    n, c, h, w = prob.size()
    return -torch.mul(prob, torch.log2(prob + 1e-30)) / np.log2(c)


def XAI_EVAL(E_sgc, full_img_gt, model, rrp_info,target_category):

    x_sgc = totensor(E_sgc)
    # x_sgc = x_sgc.cuda()
    with torch.no_grad():
        y_pred_sgc = model(x_sgc)
        y_pred_sgc = unpad_resize(y_pred_sgc,rrp_info)

    mask_tensor_sgc = y_pred_sgc[0,...]
    mask_sgc = y_pred_sgc[0,...].cpu().numpy().transpose(1,2,0)
    target_mask_f = np.float32(mask_sgc[:,:,target_category]) * full_img_gt
    target_Confidence_score = target_mask_f[np.nonzero(target_mask_f)]
    target_Confidence_score_sgc = np.mean(target_Confidence_score)

    logist_softmax_entropy_sgc = prob_2_entropy(y_pred_sgc)
    target_entropy_mask_sgc = logist_softmax_entropy_sgc[0, :, :, :].detach().cpu().numpy().transpose(1,2,0)
    target_entropy_mask_class_sgc = target_entropy_mask_sgc[:,:,target_category] * full_img_gt
    target_entropy_mask_class_sgc_sc = (np.mean(target_entropy_mask_class_sgc))
    return target_Confidence_score_sgc, target_entropy_mask_class_sgc_sc


images_dir = r'./dataset/images'
gt_dir = r'./dataset/gt'





ticks = np.linspace(0, 1, 6, endpoint=True)
classes_cmap, scale_fig, fonts, scatter_size = defaultScales()




decoder_idx = 2
thres = 0.4
n_xai = 6
n_imgs = 1
target_layers =  [model.decoder.blocks[decoder_idx - 1]]
target_category = 0
XAI_method_t3 = ["grad_cam", "grad_cam_pp", "x_grad_cam","score_cam", "eigen_cam"]

image_weight = 0.006

directory_images = os.fsencode(images_dir)
directory_gt = os.fsencode(gt_dir)


number_testing_images = 0

Model_Seg_Score_t3 = np.zeros((n_imgs))
Model_Seg_Entropy_t3 = np.zeros((n_imgs))

Seg_Score_M1_t3 = np.zeros((n_xai, n_imgs))
Seg_Entropy_M1_t3 = np.zeros((n_xai, n_imgs))

Seg_Score_M2_t3 = np.zeros((n_xai, n_imgs))
Seg_Entropy_M2_t3 = np.zeros((n_xai, n_imgs))

Seg_Score_M3_t3 = np.zeros((n_xai, n_imgs))
Seg_Entropy_M3_t3 = np.zeros((n_xai, n_imgs))

counter = 0


for (file_img, file_gt) in zip(os.listdir(directory_images), os.listdir(directory_gt) ):

    print(file_img, file_gt)
    if file_img == file_gt:
        number_testing_images = number_testing_images + 1
        print("Testing Image: ", number_testing_images)
        filename = os.fsdecode(file_img)
        filename_gt = os.fsdecode(file_gt)
        if filename.endswith(".png") and filename_gt.endswith(".png"):
            raster_file = rio.open(f'{images_dir}/{filename}')
            full_img = raster_file.read().transpose(1,2,0)
            full_img,rrp_info = ratio_resize_pad(full_img, ratio = None)
            full_img_rgba = full_img
            print("Input Image Shape: ", full_img.shape)
            if full_img.shape[-1]==4:full_img = cv2.cvtColor(full_img,cv2.COLOR_RGBA2RGB) #WHU images are RGBA
            # read gt mask
            raster_file_gt = rio.open(f'{gt_dir}/{filename_gt}')
            full_img_gt = raster_file_gt.read().transpose(1,2,0)
            full_img_gt,rrp_info_gt = ratio_resize_pad(full_img_gt, ratio = None)
            full_img_gt = np.float32(full_img_gt) / np.max(full_img_gt)
            print("GT mask shape: ", full_img_gt.shape)


            full_img = normalize(full_img)
            x = totensor(full_img)
            # x = x.cuda()

            with torch.no_grad():
                y_pred = model(x)
                y_pred = unpad_resize(y_pred,rrp_info)



            mask_tensor = y_pred[0,...]
            print('predicated tensor shape: ', mask_tensor.shape)
            mask = y_pred[0,...].cpu().numpy().transpose(1,2,0)

            target_mask_float = np.float32(mask[:,:,target_category]) * full_img_gt
            target_Confidence_score = target_mask_float[np.nonzero(target_mask_float)]
            target_Confidence_score = np.mean(target_Confidence_score)
            print("Model Target Confidence Score: ", target_Confidence_score)

            logist_softmax_entropy = prob_2_entropy(y_pred)
            target_entropy_mask = logist_softmax_entropy[0, :, :, :].detach().cpu().numpy().transpose(1,2,0)
            target_entropy_mask_class = target_entropy_mask[:,:,target_category] * full_img_gt
            model_entropy = np.mean(target_entropy_mask_class)
            print("Model Target Entropy Score: ", model_entropy )


            targets = [SemanticSegmentationTarget(target_category, target_mask_float)]

            nan_condition = np.count_nonzero(target_mask_float)
            if nan_condition != 0:

                print('running seg-grad-cam...')
                with GRADCAM_Extensions(extension = XAI_method_t3[0], model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as cam:
                    grayscale_cam_EX = cam(input_tensor=x, targets=targets)[0, :]
                    [Exmap_sgc_t3, heatmap_sgc_rgba] = show_cam_image_whu(full_img_rgba, grayscale_cam_EX, image_weight)

                    # Evaluation
                    im_bw_sgc = cv2.threshold(grayscale_cam_EX, thres, 1, cv2.THRESH_BINARY)[1]
                    # M1: Background Only
                    E_sgc_M1 = full_img * np.logical_not(im_bw_sgc)[..., None]
                    [confidence_sgc_M1_t3, entropy_sgc_M1_t3] = XAI_EVAL(E_sgc_M1, full_img_gt, model, rrp_info,target_category)
                    print("M1 -- SGC Target Confidence Score: ", confidence_sgc_M1_t3," Entropy Score: ", entropy_sgc_M1_t3)

                    # M2: Highlighted Only
                    E_sgc_M2 = full_img * im_bw_sgc[..., None]
                    [confidence_sgc_M2_t3, entropy_sgc_M2_t3] = XAI_EVAL(E_sgc_M2, full_img_gt, model, rrp_info,target_category)
                    print("M2 -- SGC Target Confidence Score: ", confidence_sgc_M2_t3," Entropy Score: ", entropy_sgc_M2_t3)

                    # M3 : Highlighted + GT
                    union_gt_sgc = np.ma.mask_or(full_img_gt,im_bw_sgc)
                    E_sgc_M3_t3 = full_img * union_gt_sgc[..., None]

                    [confidence_sgc_M3_t3, entropy_sgc_M3_t3] = XAI_EVAL(E_sgc_M3_t3, full_img_gt, model, rrp_info,target_category)
                    print("M3 -- SGC Target Confidence Score: ", confidence_sgc_M3_t3," Entropy Score: ", entropy_sgc_M3_t3)
                #______________________________
                print('running GradCam++ ...')
                with GRADCAM_Extensions(extension = XAI_method_t3[1], model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as campp:
                    grayscale_cam_EX_Plusplus = campp(input_tensor=x, targets=targets)[0, :]
                    [Exmap_sgcpp_t3, heatmap_sgcpp_rgba] = show_cam_image_whu(full_img_rgba, grayscale_cam_EX_Plusplus, image_weight)

                		# Evaluation
                    im_bw_sgcpp = cv2.threshold(grayscale_cam_EX_Plusplus, thres, 1, cv2.THRESH_BINARY)[1]

                    # M1: Background Only
                    E_sgcpp_M1 = full_img * np.logical_not(im_bw_sgcpp)[..., None]
                    [confidence_sgcpp_M1_t3, entropy_sgcpp_M1_t3] = XAI_EVAL(E_sgcpp_M1, full_img_gt, model, rrp_info,target_category)
                    print("M1 -- SGC++ Target Confidence Score: ", confidence_sgcpp_M1_t3," Entropy Score: ", entropy_sgcpp_M1_t3)

										# M2: Highlighted Only
                    E_sgcpp_M2 = full_img * im_bw_sgcpp[..., None]
                    [confidence_sgcpp_M2_t3, entropy_sgcpp_M2_t3] = XAI_EVAL(E_sgcpp_M2, full_img_gt, model, rrp_info,target_category)
                    print("M2 -- SGC++ Target Confidence Score: ", confidence_sgcpp_M2_t3," Entropy Score: ", entropy_sgcpp_M2_t3)

              			# M3: Highlighted + GT
                    union_gt_sgcpp = np.ma.mask_or(full_img_gt,im_bw_sgcpp)
                    E_sgcpp_M3_t3 = full_img * union_gt_sgcpp[..., None]
                    [confidence_sgcpp_M3_t3, entropy_sgcpp_M3_t3] = XAI_EVAL(E_sgcpp_M3_t3, full_img_gt, model, rrp_info,target_category)
                    print("M3 -- SGC++ Target Confidence Score: ", confidence_sgcpp_M3_t3," Entropy Score: ", entropy_sgcpp_M3_t3)

								#___________________________
                print('running XGradCam...')
                with GRADCAM_Extensions(extension = XAI_method_t3[2], model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as xcam:
                    grayscale_cam_EX_X = xcam(input_tensor=x, targets=targets)[0, :]
                    [Exmap_Xsgc_t3, heatmap_Xsgc_rgba] = show_cam_image_whu(full_img_rgba, grayscale_cam_EX_X, image_weight)
										# Evaluation
                    im_bw_Xsgc = cv2.threshold(grayscale_cam_EX_X, thres, 1, cv2.THRESH_BINARY)[1]

                    # M1: Background Only
                    E_Xsgc_M1 = full_img * np.logical_not(im_bw_Xsgc)[..., None]
                    [confidence_Xsgc_M1_t3, entropy_Xsgc_M1_t3] = XAI_EVAL(E_Xsgc_M1, full_img_gt, model, rrp_info,target_category)
                    print("M1 -- XSGC Target Confidence Score: ", confidence_Xsgc_M1_t3," Entropy Score: ", entropy_Xsgc_M1_t3)

										# M2: Highlighted Only
                    E_Xsgc_M2 = full_img * im_bw_Xsgc[..., None]
                    [confidence_Xsgc_M2_t3, entropy_Xsgc_M2_t3] = XAI_EVAL(E_Xsgc_M2, full_img_gt, model, rrp_info,target_category)
                    print("M2 -- XSGC Target Confidence Score: ", confidence_Xsgc_M2_t3," Entropy Score: ", entropy_Xsgc_M2_t3)

              			# M3: Highlighted + GT
                    union_gt_Xsgc = np.ma.mask_or(full_img_gt,im_bw_Xsgc)
                    E_Xsgc_M3_t3 = full_img * union_gt_Xsgc[..., None]
                    [confidence_Xsgc_M3_t3, entropy_Xsgc_M3_t3] = XAI_EVAL(E_Xsgc_M3_t3, full_img_gt, model, rrp_info,target_category)
                    print("M3 -- XSGC Target Confidence Score: ", confidence_Xsgc_M3_t3," Entropy Score: ", entropy_Xsgc_M3_t3)

								#___________________________
                print('running ScoreCam...')
                with GRADCAM_Extensions(extension = XAI_method_t3[3], model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as scorecam:
                    grayscale_cam_EX_Score = scorecam(input_tensor=x, targets=targets)[0, :]
                    [Exmap_scoresgc_t3, heatmap_scoresgc_rgba] = show_cam_image_whu(full_img_rgba, grayscale_cam_EX_Score, image_weight)

                    # Evaluation
                    im_bw_ssgc = cv2.threshold(grayscale_cam_EX_Score, thres, 1, cv2.THRESH_BINARY)[1]

                    # M1: Background Only
                    E_ssgc_M1 = full_img * np.logical_not(im_bw_ssgc)[..., None]
                    [confidence_ssgc_M1_t3, entropy_ssgc_M1_t3] = XAI_EVAL(E_ssgc_M1, full_img_gt, model, rrp_info,target_category)
                    print("M1 -- Score SGC Target Confidence Score: ", confidence_ssgc_M1_t3," Entropy Score: ", entropy_ssgc_M1_t3)

										# M2: Highlighted Only
                    E_ssgc_M2 = full_img * im_bw_ssgc[..., None]
                    [confidence_ssgc_M2_t3, entropy_ssgc_M2_t3] = XAI_EVAL(E_ssgc_M2, full_img_gt, model, rrp_info,target_category)
                    print("M2 -- Score SGC Target Confidence Score: ", confidence_ssgc_M2_t3," Entropy Score: ", entropy_ssgc_M2_t3)

              			# M3: Highlighted + GT
                    union_gt_ssgc = np.ma.mask_or(full_img_gt,im_bw_ssgc)
                    E_ssgc_M3_t3 = full_img * union_gt_ssgc[..., None]
                    [confidence_ssgc_M3_t3, entropy_ssgc_M3_t3] = XAI_EVAL(E_ssgc_M3_t3, full_img_gt, model, rrp_info,target_category)
                    print("M3 -- Score SGC Target Confidence Score: ", confidence_ssgc_M3_t3," Entropy Score: ", entropy_ssgc_M3_t3)

								#__________________________
                print('running EigenCam...')
                with GRADCAM_Extensions(extension = XAI_method_t3[4], model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as ecam:
                    grayscale_cam_EX_eigen = ecam(input_tensor=x, targets=targets)[0, :]
                    [Exmap_eigensgc_t3, heatmap_eigensgc_rgba] = show_cam_image_whu(full_img_rgba, grayscale_cam_EX_eigen, image_weight)

										# Evaluation
                    im_bw_esgc = cv2.threshold(grayscale_cam_EX_eigen, thres, 1, cv2.THRESH_BINARY)[1]

                    # M1: Background Only
                    E_esgc_M1 = full_img * np.logical_not(im_bw_esgc)[..., None]
                    [confidence_esgc_M1_t3, entropy_esgc_M1_t3] = XAI_EVAL(E_esgc_M1, full_img_gt, model, rrp_info,target_category)
                    print("M1 -- Eigen SGC Target Confidence Score: ", confidence_esgc_M1_t3," Entropy Score: ", entropy_esgc_M1_t3)

										# M2: Highlighted Only
                    E_esgc_M2 = full_img * im_bw_esgc[..., None]
                    [confidence_esgc_M2_t3, entropy_esgc_M2_t3] = XAI_EVAL(E_esgc_M2, full_img_gt, model, rrp_info,target_category)
                    print("M2 -- Eigen SGC Target Confidence Score: ", confidence_esgc_M2_t3," Entropy Score: ", entropy_esgc_M2_t3)

              			# M3: Highlighted + GT
                    union_gt_esgc = np.ma.mask_or(full_img_gt,im_bw_esgc)
                    E_esgc_M3_t3 = full_img * union_gt_esgc[..., None]
                    [confidence_esgc_M3_t3, entropy_esgc_M3_t3] = XAI_EVAL(E_esgc_M3_t3, full_img_gt, model, rrp_info,target_category)
                    print("M3 -- Eigen SGC Target Confidence Score: ", confidence_esgc_M3_t3," Entropy Score: ", entropy_esgc_M3_t3)

                Model_Seg_Score_t3[counter] = target_Confidence_score
                Model_Seg_Entropy_t3[counter] = model_entropy

                Seg_Score_M1_t3[0,counter] = confidence_sgc_M1_t3
                Seg_Score_M1_t3[1,counter] = confidence_sgcpp_M1_t3
                Seg_Score_M1_t3[2,counter] = confidence_Xsgc_M1_t3
                Seg_Score_M1_t3[3,counter] = confidence_ssgc_M1_t3
                Seg_Score_M1_t3[4,counter] = confidence_esgc_M1_t3

                Seg_Entropy_M1_t3[0,counter] = entropy_sgc_M1_t3
                Seg_Entropy_M1_t3[1,counter] = entropy_sgcpp_M1_t3
                Seg_Entropy_M1_t3[2,counter] = entropy_Xsgc_M1_t3
                Seg_Entropy_M1_t3[3,counter] = entropy_ssgc_M1_t3
                Seg_Entropy_M1_t3[4,counter] = entropy_esgc_M1_t3


                Seg_Score_M2_t3[0,counter] = confidence_sgc_M2_t3
                Seg_Score_M2_t3[1,counter] = confidence_sgcpp_M2_t3
                Seg_Score_M2_t3[2,counter] = confidence_Xsgc_M2_t3
                Seg_Score_M2_t3[3,counter] = confidence_ssgc_M2_t3
                Seg_Score_M2_t3[4,counter] = confidence_esgc_M2_t3

                Seg_Entropy_M2_t3[0,counter] = entropy_sgc_M2_t3
                Seg_Entropy_M2_t3[1,counter] = entropy_sgcpp_M2_t3
                Seg_Entropy_M2_t3[2,counter] = entropy_Xsgc_M2_t3
                Seg_Entropy_M2_t3[3,counter] = entropy_ssgc_M2_t3
                Seg_Entropy_M2_t3[4,counter] = entropy_esgc_M2_t3

                Seg_Score_M3_t3[0,counter] = confidence_sgc_M3_t3
                Seg_Score_M3_t3[1,counter] = confidence_sgcpp_M3_t3
                Seg_Score_M3_t3[2,counter] = confidence_Xsgc_M3_t3
                Seg_Score_M3_t3[3,counter] = confidence_ssgc_M3_t3
                Seg_Score_M3_t3[4,counter] = confidence_esgc_M3_t3

                Seg_Entropy_M3_t3[0,counter] = entropy_sgc_M3_t3
                Seg_Entropy_M3_t3[1,counter] = entropy_sgcpp_M3_t3
                Seg_Entropy_M3_t3[2,counter] = entropy_Xsgc_M3_t3
                Seg_Entropy_M3_t3[3,counter] = entropy_ssgc_M3_t3
                Seg_Entropy_M3_t3[4,counter] = entropy_esgc_M3_t3


                counter = counter + 1

                if(number_testing_images == 2):
                    break



                fig = plt.figure(figsize=(8 * scale_fig, 8 * scale_fig))

                plt.subplot(3,3,1)
                plt.axis('off')
                plt.imshow(full_img_rgba)
                plt.title('Input image', fontsize=fonts)

                plt.subplot(3,3,2)
                plt.axis('off')
                plt.imshow(full_img_gt, cmap='gray', vmin=0, vmax=1)
                plt.title('Ground Truth (GT) Mask', fontsize=fonts)

                plt.subplot(3,3,3)
                plt.axis('off')
                plt.imshow(target_mask_float,  cmap='gray', vmin=0, vmax=1)
                plt.title('Predicted Mask', fontsize=fonts)

                plt.subplot(3,3,4)
                plt.axis('off')
                plt.imshow(Exmap_sgc_t3)
                plt.title('Seg-Grad-CAM', fontsize=fonts)

                plt.subplot(3,3,5)
                plt.axis('off')
                plt.imshow(Exmap_sgcpp_t3)
                plt.title('Seg-Grad-CAM ++', fontsize=fonts)

                plt.subplot(3,3,6)
                plt.axis('off')
                plt.imshow(Exmap_Xsgc_t3)
                plt.title('Seg-XGrad-CAM', fontsize=fonts)

                plt.subplot(3,3,7)
                plt.axis('off')
                plt.imshow(Exmap_scoresgc_t3)
                plt.title('Seg-Score-CAM', fontsize=fonts)

                plt.subplot(3,3,8)
                plt.axis('off')
                plt.imshow(Exmap_eigensgc_t3)
                plt.title('Seg-Eigen-CAM', fontsize=fonts)

                fig.savefig('./results/SegGradCam_Extensions_t3_{}_{}.png'.format(filename,decoder_idx), bbox_inches='tight', pad_inches=0)


                fig = plt.figure(figsize=(8 * scale_fig, 8 * scale_fig))

                plt.subplot(3,3,1)
                plt.axis('off')
                plt.imshow(full_img_rgba)
                plt.title('Input image', fontsize=fonts)

                plt.subplot(3,3,2)
                plt.axis('off')
                plt.imshow(full_img_gt, cmap='gray', vmin=0, vmax=1)
                plt.title('Ground Truth (GT) Mask', fontsize=fonts)

                plt.subplot(3,3,3)
                plt.axis('off')
                plt.imshow(target_mask_float,  cmap='gray', vmin=0, vmax=1)
                plt.title('Predicted Mask', fontsize=fonts)

                plt.subplot(3,3,4)
                plt.axis('off')
                plt.imshow(E_Xsgc_M3_t3)
                plt.title('Seg-Grad-CAM', fontsize=fonts)

                plt.subplot(3,3,5)
                plt.axis('off')
                plt.imshow(E_sgcpp_M3_t3)
                plt.title('Seg-Grad-CAM ++', fontsize=fonts)

                plt.subplot(3,3,6)
                plt.axis('off')
                plt.imshow(E_Xsgc_M3_t3)
                plt.title('Seg-XGrad-CAM', fontsize=fonts)

                plt.subplot(3,3,7)
                plt.axis('off')
                plt.imshow(E_ssgc_M3_t3)
                plt.title('Seg-Score-CAM', fontsize=fonts)

                plt.subplot(3,3,8)
                plt.axis('off')
                plt.imshow(E_esgc_M3_t3)
                plt.title('Seg-Eigen-CAM', fontsize=fonts)

                fig.savefig('./results/SegGradCam_Modified_M3_t3_{}_{}.png'.format(filename,decoder_idx), bbox_inches='tight', pad_inches=0)


Output hidden; open in https://colab.research.google.com to view.

## Repeating Task 1

In [25]:
results_t3 = {
    method: {
        th: {'M1': {}, 'M2': {}, 'M3': {}} for th in thresholds
    }
    for method in XAI_method_t3
}

for method_name in XAI_method_t3:
    print(f'Running {method_name} ...')

    with GRADCAM_Extensions(extension=method_name, model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as cam:
        grayscale_cam = cam(input_tensor=x, targets=targets)[0, :]

        for thres in thresholds:
            im_bw = cv2.threshold(grayscale_cam, thres, 1, cv2.THRESH_BINARY)[1]

            # M1: Background Only
            E_M1 = full_img * np.logical_not(im_bw)[..., None]
            confidence_M1, entropy_M1 = XAI_EVAL(E_M1, full_img_gt, model, rrp_info, target_category)

            # M2: Highlighted Only
            E_M2 = full_img * im_bw[..., None]
            confidence_M2, entropy_M2 = XAI_EVAL(E_M2, full_img_gt, model, rrp_info, target_category)

            # M3: Highlighted + GT
            union_mask = np.ma.mask_or(full_img_gt, im_bw)
            E_M3 = full_img * union_mask[..., None]
            confidence_M3, entropy_M3 = XAI_EVAL(E_M3, full_img_gt, model, rrp_info, target_category)

            results_t3[method_name][thres]['M1'] = {'confidence': confidence_M1, 'entropy': entropy_M1}
            results_t3[method_name][thres]['M2'] = {'confidence': confidence_M2, 'entropy': entropy_M2}
            results_t3[method_name][thres]['M3'] = {'confidence': confidence_M3, 'entropy': entropy_M3}


Running grad_cam ...
Running grad_cam_pp ...
Running x_grad_cam ...
Running score_cam ...


100%|██████████| 16/16 [05:16<00:00, 19.78s/it]


Running eigen_cam ...


**Printing Results**

In [26]:
rows = []
for method in results_t3:
    for th in results_t3[method]:
        for metric in results_t3[method][th]:
            row = {
                'Method': method,
                'Threshold': th,
                'Metric': metric,
                'Confidence': results_t3[method][th][metric]['confidence'],
                'Entropy': results_t3[method][th][metric]['entropy']
            }
            rows.append(row)

df_results_t3 = pd.DataFrame(rows)

df_results_t3 = df_results_t3.sort_values(by=['Method', 'Threshold', 'Metric'])

print(df_results_t3.to_string(index=False))


     Method  Threshold Metric  Confidence  Entropy
  eigen_cam        0.1     M1    0.030938 0.000691
  eigen_cam        0.1     M2    0.000021 0.000032
  eigen_cam        0.1     M3    0.168842 0.003327
  eigen_cam        0.2     M1    0.089913 0.002056
  eigen_cam        0.2     M2    0.000022 0.000033
  eigen_cam        0.2     M3    0.048698 0.001535
  eigen_cam        0.3     M1    0.188565 0.003024
  eigen_cam        0.3     M2    0.000323 0.000078
  eigen_cam        0.3     M3    0.022349 0.001169
  eigen_cam        0.4     M1    0.358685 0.003887
  eigen_cam        0.4     M2    0.173499 0.003378
  eigen_cam        0.4     M3    0.038197 0.001090
  eigen_cam        0.5     M1    0.466749 0.004320
  eigen_cam        0.5     M2    0.744334 0.003155
  eigen_cam        0.5     M3    0.021010 0.000897
   grad_cam        0.1     M1    0.011093 0.000573
   grad_cam        0.1     M2    0.040254 0.000425
   grad_cam        0.1     M3    0.027777 0.001038
   grad_cam        0.2     M1  

**Best XAI Method for each Eval Metric and threshold**

In [27]:
best_methods_t3 = []

for (thresh, metric), group in df_results_t3.groupby(['Threshold', 'Metric']):
    if metric == 'M1':
        # Lowest confidence
        best_row = group.loc[group['Confidence'].idxmin()]
    elif metric == 'M2':
        # Highest confidence
        best_row = group.loc[group['Confidence'].idxmax()]
    elif metric == 'M3':
        # Lowest entropy
        best_row = group.loc[group['Entropy'].idxmin()]
    else:
        continue

    best_methods_t3.append(best_row)

best_methods_df_t3 = pd.DataFrame(best_methods_t3)

best_methods_df_t3 = best_methods_df_t3.sort_values(['Metric', 'Threshold']).reset_index(drop=True)
print(best_methods_df_t3[['Threshold', 'Metric', 'Method', 'Confidence', 'Entropy']])


    Threshold Metric       Method  Confidence   Entropy
0         0.1     M1  grad_cam_pp    0.000021  0.000032
1         0.2     M1  grad_cam_pp    0.000282  0.000066
2         0.3     M1  grad_cam_pp    0.003197  0.000232
3         0.4     M1  grad_cam_pp    0.058054  0.001856
4         0.5     M1  grad_cam_pp    0.206682  0.004750
5         0.1     M2    score_cam    0.737812  0.001414
6         0.2     M2    score_cam    0.974102  0.001188
7         0.3     M2    score_cam    0.993267  0.000493
8         0.4     M2    score_cam    0.984366  0.000901
9         0.5     M2    score_cam    0.980316  0.001014
10        0.1     M3    score_cam    0.039112  0.000827
11        0.2     M3   x_grad_cam    0.015666  0.000609
12        0.3     M3  grad_cam_pp    0.017148  0.000625
13        0.4     M3  grad_cam_pp    0.015107  0.000584
14        0.5     M3  grad_cam_pp    0.015407  0.000705


**Overall Best XAI Method**

In [28]:
method_score_t3 = best_methods_df_t3['Method'].value_counts()

print("Method Selection Frequency:")
print(method_score_t3)

best_overall_method_t3 = method_score_t3.idxmax()
print(f"\nBest overall XAI method: {best_overall_method_t3}")


Method Selection Frequency:
Method
grad_cam_pp    8
score_cam      6
x_grad_cam     1
Name: count, dtype: int64

Best overall XAI method: grad_cam_pp


**Best Threshold Value(s)**

In [29]:
def normalize(arr):
    return (arr - np.min(arr)) / (np.max(arr) - np.min(arr))

best_thresholds = {}

for method in df_results_t3['Method'].unique():
    df_m = df_results_t3[df_results_t3['Method'] == method]

    conf_pivot = df_m.pivot(index='Threshold', columns='Metric', values='Confidence')
    entropy_pivot = df_m.pivot(index='Threshold', columns='Metric', values='Entropy')

    m1_conf_norm = normalize(conf_pivot['M1'].values)
    m2_conf_norm = normalize(conf_pivot['M2'].values)
    m3_entropy_norm = normalize(entropy_pivot['M3'].values)

    # Combined score: low M1 conf, high M2 conf, low M3 entropy
    combined_score = m1_conf_norm + (1 - m2_conf_norm) + m3_entropy_norm

    best_idx = np.argmin(combined_score)
    best_thresh = conf_pivot.index[best_idx]

    best_thresholds[method] = {
        'best_threshold': best_thresh,
        'combined_scores': dict(zip(conf_pivot.index, combined_score))
    }

for method, info in best_thresholds.items():
    print(f"Method: {method}")
    print(f" Best threshold: {info['best_threshold']}")
    print()


Method: eigen_cam
 Best threshold: 0.5

Method: grad_cam
 Best threshold: 0.4

Method: grad_cam_pp
 Best threshold: 0.4

Method: score_cam
 Best threshold: 0.3

Method: x_grad_cam
 Best threshold: 0.4



**Comparing results of Task 1 and Task 3**

In [30]:
def flatten_results(results):
    rows = []
    for method, th_dict in results.items():
        for th, metrics in th_dict.items():
            for mi, values in metrics.items():
                rows.append({
                    'Method': method,
                    'Threshold': th,
                    'Metric': mi,
                    'Confidence': values['confidence'],
                    'Entropy': values['entropy']
                })
    return pd.DataFrame(rows)

In [31]:
df_t1 = flatten_results(results_t1)
df_t3 = flatten_results(results_t3)

df_compare = df_t1.merge(df_t3, on=['Method', 'Threshold', 'Metric'], suffixes=('_t1', '_t3'))

method_summary = {}

for method in df_compare['Method'].unique():
    method_df = df_compare[df_compare['Method'] == method]
    improved_metrics = {'M1': 0, 'M2': 0, 'M3': 0}
    total_counts = {'M1': 0, 'M2': 0, 'M3': 0}

    for _, row in method_df.iterrows():
        metric = row['Metric']
        total_counts[metric] += 1

        if metric == 'M1':
            if row['Confidence_t3'] < row['Confidence_t1']:
                improved_metrics['M1'] += 1
        elif metric == 'M2':
            if row['Confidence_t3'] > row['Confidence_t1']:
                improved_metrics['M2'] += 1
        elif metric == 'M3':
            if row['Entropy_t3'] < row['Entropy_t1']:
                improved_metrics['M3'] += 1

    method_summary[method] = {
        'M1_Improved': f"{improved_metrics['M1']}/{total_counts['M1']}",
        'M2_Improved': f"{improved_metrics['M2']}/{total_counts['M2']}",
        'M3_Improved': f"{improved_metrics['M3']}/{total_counts['M3']}",
    }

for method, scores in method_summary.items():
    print(f"XAI Method: {method}")
    print(f"  M1 (Background Only - ↓ Conf): {scores['M1_Improved']} improved")
    print(f"  M2 (Highlighted Only - ↑ Conf): {scores['M2_Improved']} improved")
    print(f"  M3 (Highlight+GT - ↓ Entropy): {scores['M3_Improved']} improved")
    print("  --> Overall Improvement:", "Yes" if any(int(x.split('/')[0]) > 0 for x in scores.values()) else "No")
    print("-" * 50)


XAI Method: grad_cam
  M1 (Background Only - ↓ Conf): 5/5 improved
  M2 (Highlighted Only - ↑ Conf): 3/5 improved
  M3 (Highlight+GT - ↓ Entropy): 5/5 improved
  --> Overall Improvement: Yes
--------------------------------------------------
XAI Method: grad_cam_pp
  M1 (Background Only - ↓ Conf): 5/5 improved
  M2 (Highlighted Only - ↑ Conf): 3/5 improved
  M3 (Highlight+GT - ↓ Entropy): 4/5 improved
  --> Overall Improvement: Yes
--------------------------------------------------
XAI Method: x_grad_cam
  M1 (Background Only - ↓ Conf): 5/5 improved
  M2 (Highlighted Only - ↑ Conf): 3/5 improved
  M3 (Highlight+GT - ↓ Entropy): 5/5 improved
  --> Overall Improvement: Yes
--------------------------------------------------
XAI Method: score_cam
  M1 (Background Only - ↓ Conf): 1/5 improved
  M2 (Highlighted Only - ↑ Conf): 4/5 improved
  M3 (Highlight+GT - ↓ Entropy): 5/5 improved
  --> Overall Improvement: Yes
--------------------------------------------------
XAI Method: eigen_cam
  M1

## Repeating Task 2

In [32]:
thres = 0.4

In [33]:
drop_results_t3 = {method: {} for method in XAI_method_t3}

for method_name in XAI_method_t3:
    print(f'running {method_name} ...')

    with GRADCAM_Extensions(extension=method_name, model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as cam:
        grayscale_cam = cam(input_tensor=x, targets=targets)[0, :]

        im_bw = cv2.threshold(grayscale_cam, thres, 1, cv2.THRESH_BINARY)[1]

        # M3: Highlighted + GT
        union_mask = np.ma.mask_or(full_img_gt, im_bw)
        E_img = full_img * union_mask[..., None]
        drop_result_t3 = XAI_EVAL_Drop(E_img, full_img_gt, model, full_img, rrp_info, target_category)

        print(f"{method_name} Drop in Segmentation: ", drop_result_t3)
        drop_results_t3[method_name] = {
            'drop': drop_result_t3
        }


running grad_cam ...
grad_cam Drop in Segmentation:  -0.28235038241062205
running grad_cam_pp ...
grad_cam_pp Drop in Segmentation:  -0.2809565167789135
running x_grad_cam ...
x_grad_cam Drop in Segmentation:  -0.28192515682477465
running score_cam ...


100%|██████████| 16/16 [05:08<00:00, 19.29s/it]


score_cam Drop in Segmentation:  -0.27227709757772023
running eigen_cam ...
eigen_cam Drop in Segmentation:  -0.23373582717876795


In [34]:
ranked_methods_t3 = sorted(drop_results_t3.items(), key=lambda x: x[1]['drop'])

print("XAI Methods Ranked by Faithfulness to the model’s true reasoning:")
for i, (method, result) in enumerate(ranked_methods_t3, start=1):
    print(f"{i}. {method}: Drop = {result['drop']:.4f}")


XAI Methods Ranked by Faithfulness to the model’s true reasoning:
1. grad_cam: Drop = -0.2824
2. x_grad_cam: Drop = -0.2819
3. grad_cam_pp: Drop = -0.2810
4. score_cam: Drop = -0.2723
5. eigen_cam: Drop = -0.2337


In [35]:
best_method_t3, best_result_t3 = ranked_methods_t3[0]

print(f"\n Best XAI method (lowest drop in IoU): {best_method_t3} with drop = {best_result_t3['drop']:.4f}")



 Best XAI method (lowest drop in IoU): grad_cam with drop = -0.2824


Comparing results of Task 2 and Task 3

In [36]:
if best_result_t3['drop'] < best_result_t2['drop']:
    trend = "decreased (improved)"
elif best_result_t3['drop'] > best_result_t2['drop']:
    trend = "increased (worsened)"
else:
    trend = "unchanged"

print(f"IoU drop has {trend} in Task 3 compared to Task 2.")


IoU drop has increased (worsened) in Task 3 compared to Task 2.


---
Incorporating a second decoder can modify the model’s segmentation outputs and internal feature learning, thereby impacting the clarity, reliability, and stability of the XAI explanations.